In [42]:
import requests
from urllib.parse import urlparse
import urllib.request
import os
from dotenv import load_dotenv
import json
from PIL import Image
from io import BytesIO

load_dotenv()

SERP_API_KEY = os.getenv("SERP_API_KEY")

def download_images(query, num_images=20):
    # Replace spaces with '+' for the URL
    search_query = query.replace(" ", "+")

    dir_to_save = f"downloaded_images/{query.replace(' ', '_')}"

    # Create the directory if it doesn't exist
    if not os.path.exists(dir_to_save):
        os.makedirs(dir_to_save)

    # Check how many images are already downloaded
    count = len(os.listdir(dir_to_save))
    if count >= num_images:
        print(f"Already have {count} images for {query}, which meets or exceeds the required {num_images}.")
        return

    # Start pagination from 0
    start = 0
    images_fetched = []
    
    # Keep fetching images until we have enough or run out of results
    while len(images_fetched) < num_images and start < 1000:  # Limit to avoid infinite loops
        url = f"https://serpapi.com/search.json?engine=google_images&q={search_query}+pokemon&api_key={SERP_API_KEY}&start={start}"
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Failed to fetch images from SerpAPI: Status code {response.status_code}")
            break
        
        json_data = json.loads(response.text)
        
        # Check if there are any images in the response
        if 'images_results' not in json_data:
            print("No more images found.")
            break
        
        images = json_data['images_results']
        if not images:
            print("No more images available.")
            break
        
        print(f"Fetched {len(images)} images starting at position {start} for query '{query}'")
        images_fetched.extend(images)
        start += len(images)  # Move to the next page of results

    print(f"Total images fetched for {query}: {len(images_fetched)}")

    # Download until we reach num_images
    for i, img in enumerate(images_fetched):
        if count >= num_images:
            break
        try:
            img_url = img['original']
            if img_url and img_url.startswith("http"):
                # File name based on count
                file_path = os.path.join(dir_to_save, f"{query.replace(' ', '_')}_{count+1}.jpg")
                if download_single_image(img_url, file_path):
                    count += 1
        except Exception as e:
            print(f"Error downloading image {i+1}: {e}")

    if count < num_images:
        print(f"Only downloaded {count} images instead of {num_images} for query '{query}'.")

def get_all_pokemon():
    url = "https://pokeapi.co/api/v2/pokemon/?limit=151"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to fetch Pokémon data: Status code {response.status_code}")
        return []
    
    json_data = json.loads(response.text)
    results = json_data['results']
    return results

# Function to download a single image
def download_single_image(url, file_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://www.google.com/"
    }
    
    try:
        response = requests.get(url, headers=headers, stream=True)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))

            # Convert to RGB to handle formats like PNG that may have alpha channels
            if image.mode in ("RGBA", "P"):  # If the image has an alpha channel
                image = image.convert("RGB")

            image.save(file_path, "JPEG")
            print(f"Saved: {file_path}")
            return True
        else:
            print(f"Failed with status code: {response.status_code} - {response.reason}")
            return False
    except Exception as e:
        print(f"Error processing image: {e}")
        return False

In [43]:
all_pokemons = get_all_pokemon()
for pokemon in all_pokemons:
    pokemon_name = pokemon['name']
    download_images(pokemon_name, num_images=20)

Already have 20 images for bulbasaur, which meets or exceeds the required 20.
Already have 20 images for ivysaur, which meets or exceeds the required 20.
Already have 20 images for venusaur, which meets or exceeds the required 20.
Already have 20 images for charmander, which meets or exceeds the required 20.
Already have 20 images for charmeleon, which meets or exceeds the required 20.
Already have 20 images for charizard, which meets or exceeds the required 20.
Already have 20 images for squirtle, which meets or exceeds the required 20.
Already have 20 images for wartortle, which meets or exceeds the required 20.
Already have 20 images for blastoise, which meets or exceeds the required 20.
Already have 20 images for caterpie, which meets or exceeds the required 20.
Already have 20 images for metapod, which meets or exceeds the required 20.
Already have 20 images for butterfree, which meets or exceeds the required 20.
Already have 20 images for weedle, which meets or exceeds the requir

In [41]:
all_pokemons

[{'name': 'bulbasaur', 'url': 'https://pokeapi.co/api/v2/pokemon/1/'},
 {'name': 'ivysaur', 'url': 'https://pokeapi.co/api/v2/pokemon/2/'},
 {'name': 'venusaur', 'url': 'https://pokeapi.co/api/v2/pokemon/3/'},
 {'name': 'charmander', 'url': 'https://pokeapi.co/api/v2/pokemon/4/'},
 {'name': 'charmeleon', 'url': 'https://pokeapi.co/api/v2/pokemon/5/'},
 {'name': 'charizard', 'url': 'https://pokeapi.co/api/v2/pokemon/6/'},
 {'name': 'squirtle', 'url': 'https://pokeapi.co/api/v2/pokemon/7/'},
 {'name': 'wartortle', 'url': 'https://pokeapi.co/api/v2/pokemon/8/'},
 {'name': 'blastoise', 'url': 'https://pokeapi.co/api/v2/pokemon/9/'},
 {'name': 'caterpie', 'url': 'https://pokeapi.co/api/v2/pokemon/10/'},
 {'name': 'metapod', 'url': 'https://pokeapi.co/api/v2/pokemon/11/'},
 {'name': 'butterfree', 'url': 'https://pokeapi.co/api/v2/pokemon/12/'},
 {'name': 'weedle', 'url': 'https://pokeapi.co/api/v2/pokemon/13/'},
 {'name': 'kakuna', 'url': 'https://pokeapi.co/api/v2/pokemon/14/'},
 {'name': '